<a href="https://colab.research.google.com/github/PRASANNA-416/Human-Activity-Detection-/blob/main/HCD_TRIAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [2]:

import pandas as pd

# Load the training CSV file
train_csv_path = '/content/gdrive/MyDrive/ML/Human Action Recognition/Human Action Recognition/Final_training_set.csv'
train_data = pd.read_csv(train_csv_path)

# Load the testing CSV file
test_csv_path = '/content/gdrive/MyDrive/ML/Human Action Recognition/Human Action Recognition/Testing_set.csv'
test_data = pd.read_csv(test_csv_path)

In [4]:

from tensorflow.keras.preprocessing.image import ImageDataGenerator



# Define the target image size and batch size
target_size = (224, 224)  # Adjust according to your requirements
batch_size = 64

# Create ImageDataGenerators with preprocessing options
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,  # Normalize pixel values between 0 and 1
)

test_datagen = ImageDataGenerator(rescale=1.0/255.0)

# Create train and test generators
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_data,
    directory='/content/gdrive/MyDrive/ML/Human Action Recognition/Human Action Recognition/FINAL_TRAIN',  # Directory containing the train images
    x_col='filename',  # Column name for the filenames
    y_col='label',  # Column name for the labels
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_data,
    directory='/content/gdrive/MyDrive/ML/Human Action Recognition/Human Action Recognition/test',
    x_col='filename',  # Column name for the filenames
    y_col='label',  # Column name for the labels
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical'
)

# Check the class indices assigned by the generator
class_indices = test_generator.class_indices
print(class_indices)



Found 17566 validated image filenames belonging to 15 classes.
Found 2520 validated image filenames belonging to 15 classes.
{'calling': 0, 'clapping': 1, 'cycling': 2, 'dancing': 3, 'drinking': 4, 'eating': 5, 'fighting': 6, 'hugging': 7, 'laughing': 8, 'listening_to_music': 9, 'running': 10, 'sitting': 11, 'sleeping': 12, 'texting': 13, 'using_laptop': 14}


In [7]:
import tensorflow as tf
import tensorflow_hub as hub

# Define the model
m = tf.keras.Sequential([
    hub.KerasLayer(
        'https://tfhub.dev/google/imagenet/mobilenet_v3_small_075_224/feature_vector/5',
        trainable=False),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(15, activation='softmax')
])


# Build the model with the default input shape
m.build((None, 224, 224, 3))  # Batch input shape.

# Print the model summary
print(m.summary())


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_1 (KerasLayer)  (None, 1024)              1026552   
                                                                 
 flatten_1 (Flatten)         (None, 1024)              0         
                                                                 
 dense_3 (Dense)             (None, 64)                65600     
                                                                 
 dropout_2 (Dropout)         (None, 64)                0         
                                                                 
 dense_4 (Dense)             (None, 15)                975       
                                                                 
Total params: 1,093,127
Trainable params: 66,575
Non-trainable params: 1,026,552
_________________________________________________________________
None


In [9]:

m.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

# Calculate the steps per epoch based on the desired batch size and the size of your dataset
steps_per_epoch = len(train_generator) // batch_size

# Update the number of epochs and steps_per_epoch
m.fit(train_generator, epochs=15, steps_per_epoch=275)

Epoch 1/15
275/275 [==============================] - 3544s 13s/step - loss: 2.4865 - accuracy: 0.1989
Epoch 2/15
275/275 [==============================] - 255s 926ms/step - loss: 1.9943 - accuracy: 0.3713
Epoch 3/15
275/275 [==============================] - 259s 940ms/step - loss: 1.7593 - accuracy: 0.4332
Epoch 4/15
275/275 [==============================] - 252s 916ms/step - loss: 1.6414 - accuracy: 0.4751
Epoch 5/15
275/275 [==============================] - 251s 910ms/step - loss: 1.5605 - accuracy: 0.4940
Epoch 6/15
275/275 [==============================] - 263s 956ms/step - loss: 1.4956 - accuracy: 0.5158
Epoch 7/15
275/275 [==============================] - 254s 921ms/step - loss: 1.4484 - accuracy: 0.5323
Epoch 8/15
275/275 [==============================] - 261s 946ms/step - loss: 1.4072 - accuracy: 0.5445
Epoch 9/15
275/275 [==============================] - 251s 912ms/step - loss: 1.3775 - accuracy: 0.5548
Epoch 10/15
275/275 [==============================] - 257s 935ms

In [10]:



# Evaluate the model on the test generator
test_loss, test_accuracy = m.evaluate(test_generator)

print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

40/40 [==============================] - 602s 15s/step - loss: 1.1359 - accuracy: 0.6302
Test Loss: 1.13593590259552
Test Accuracy: 0.6301587224006653


In [11]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.layers import Input, Dense, Dropout, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the target image size and batch size
target_size = (224, 224)  # Adjust according to your requirements
batch_size = 64

# Create ImageDataGenerators with preprocessing options
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,  # Normalize pixel values between 0 and 1
)

test_datagen = ImageDataGenerator(rescale=1.0/255.0)

# Create train and test generators
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_data,
    directory='/content/gdrive/MyDrive/ML/Human Action Recognition/Human Action Recognition/FINAL_TRAIN',
    x_col='filename',  # Column name for the filenames
    y_col='label',  # Column name for the labels
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_data,
    directory='/content/gdrive/MyDrive/ML/Human Action Recognition/Human Action Recognition/test',
    x_col='filename',  # Column name for the filenames
    y_col='label',  # Column name for the labels
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical'
)

# Define the MobileNet V3 Large model
mobilenet_model = tf.keras.Sequential([
    hub.KerasLayer('https://tfhub.dev/google/imagenet/mobilenet_v3_large_075_224/feature_vector/5', trainable=False),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(15, activation='softmax')
])

# Define the EfficientNet Lite3 model
efficientnet_model = tf.keras.Sequential([
    hub.KerasLayer('https://tfhub.dev/tensorflow/efficientnet/lite3/classification/2', trainable=False),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(15, activation='softmax')
])

# Create an input layer to accept the images
input_layer = Input(shape=(target_size[0], target_size[1], 3))

# Get the output predictions from both models
mobilenet_output = mobilenet_model(input_layer)
efficientnet_output = efficientnet_model(input_layer)

# Average the predictions from both models
ensemble_output = tf.keras.layers.Average()([mobilenet_output, efficientnet_output])

# Create the ensemble model
ensemble_model = Model(inputs=input_layer, outputs=ensemble_output)

#


Found 17566 validated image filenames belonging to 15 classes.
Found 2520 validated image filenames belonging to 15 classes.


In [12]:
# Compile the model
ensemble_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print the model summary
print(ensemble_model.summary())


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 sequential_2 (Sequential)      (None, 15)           3076975     ['input_1[0][0]']                
                                                                                                  
 sequential_3 (Sequential)      (None, 15)           8547447     ['input_1[0][0]']                
                                                                                                  
 average (Average)              (None, 15)           0           ['sequential_2[0][0]',       

In [13]:

# Train the model
ensemble_model.fit(train_generator, epochs=15, steps_per_epoch=275)




Epoch 1/15
275/275 [==============================] - 2121s 8s/step - loss: 1.6570 - accuracy: 0.4695
Epoch 2/15
275/275 [==============================] - 2144s 8s/step - loss: 1.2228 - accuracy: 0.6095
Epoch 3/15
275/275 [==============================] - 2167s 8s/step - loss: 1.0597 - accuracy: 0.6618
Epoch 4/15
275/275 [==============================] - 2150s 8s/step - loss: 0.9463 - accuracy: 0.6903
Epoch 5/15
275/275 [==============================] - 2156s 8s/step - loss: 0.8559 - accuracy: 0.7175
Epoch 6/15
275/275 [==============================] - 2177s 8s/step - loss: 0.7681 - accuracy: 0.7431
Epoch 7/15
275/275 [==============================] - 2167s 8s/step - loss: 0.7113 - accuracy: 0.7614
Epoch 8/15
275/275 [==============================] - 2165s 8s/step - loss: 0.6500 - accuracy: 0.7848
Epoch 9/15
275/275 [==============================] - 2171s 8s/step - loss: 0.6071 - accuracy: 0.7959
Epoch 10/15
275/275 [==============================] - 2161s 8s/step - loss: 0.569

UnknownError: ignored

In [14]:
# Evaluate the model
# Evaluate the model on the test generator
test_loss, test_accuracy = ensemble_model.evaluate(test_generator)

print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

40/40 [==============================] - 314s 8s/step - loss: 1.1436 - accuracy: 0.7123
Test Loss: 1.143625259399414
Test Accuracy: 0.7123016119003296


In [15]:
ensemble_model.save_weights('Ensemble.h5')
from google.colab import files
files.download('Ensemble.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [16]:
import os
def get_file_size(file_path):
    size = os.path.getsize(file_path)
    return size

In [17]:

def convert_bytes(size, unit=None):
    if unit == "KB":
        return print('File size: ' + str(round(size / 1024, 3)) + ' Kilobytes')
    elif unit == "MB":
        return print('File size: ' + str(round(size / (1024 * 1024), 3)) + ' Megabytes')
    else:
        return print('File size: ' + str(size) + ' bytes')

In [18]:
TF_LITE_MODEL_FILE_NAME = "tf_lite_model.tflite"

In [19]:
tf_lite_converter = tf.lite.TFLiteConverter.from_keras_model(ensemble_model)
tf_lite_converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
# tf_lite_converter.optimizations = [tf.lite.Optimize.DEFAULT]
# tf_lite_converter.target_spec.supported_types = [tf.float16]
tflite_model = tf_lite_converter.convert()

In [20]:
tflite_model_name = TF_LITE_MODEL_FILE_NAME
open(tflite_model_name, "wb").write(tflite_model)

12454184

In [21]:
convert_bytes(get_file_size(TF_LITE_MODEL_FILE_NAME), "MB")

File size: 11.877 Megabytes
